# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [3]:
import mxnet as mx
import tensorflow as tf
import pkgutil, types, pprint
import numpy as np
from random import shuffle

def doc_dump(base, docs={}):
    try:
        for a in dir(eval(base)):
            if isinstance(eval(base).__dict__.get(a), types.FunctionType):
                doc = eval(base).__dict__.get(a).__doc__
                docs[base + '.' + a] = base + '.' + a + (doc if doc else '')
        for importer, modname, ispkg in pkgutil.walk_packages(path=eval(base).__path__,
                                                              prefix=base + '.',
                                                              onerror=lambda e: None):
            docs = doc_dump(modname, docs)
    except Exception as e:
        pass
    return docs
        
tf_docs = doc_dump('tf', {})
mx_docs = doc_dump('mx', {})

/usr/local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [6]:
SUB_SAMPLE = -1
tf_labels, tf_values = zip(*list(tf_docs.items())[:SUB_SAMPLE])
mx_labels, mx_values = zip(*list(mx_docs.items())[:SUB_SAMPLE])

In [7]:
HIDDEN_UNITS = 100
LAYERS = 1
BATCH_SIZE = 1
INPUT_SIZE = 1

from scipy import sparse
from scipy.sparse import linalg as s_linalg

W = sparse.rand(HIDDEN_UNITS, HIDDEN_UNITS, density=0.1)
W = W*np.real(0.9/max(s_linalg.eigs(W)[0]))
U = 0.8*sparse.rand(HIDDEN_UNITS, INPUT_SIZE, density=0.1)

tf_states = []
for v in tf_values:
    h = np.zeros((HIDDEN_UNITS, 1))
    for c in v:
        h = np.tanh(W.dot(h) + U*ord(c))
    tf_states.append(mx.nd.array(h))
    
mx_states = []
for v in mx_values:
    h = np.zeros((HIDDEN_UNITS, 1))
    for c in v:
        h = np.tanh(W.dot(h) + U*ord(c))
    mx_states.append(mx.nd.array(h))

In [39]:
import random

net = mx.gluon.nn.Dense(HIDDEN_UNITS)
ctx = mx.cpu()
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

trainer = mx.gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})
l2loss = mx.gluon.loss.L2Loss()

data = list(enumerate(tf_states + mx_states))

for epoch in range(100):
    avg_loss = 0.0
    with mx.autograd.record():
        loss = 0.0
        for k in range(40):
            (i, x), (j, y) = random.choice(data), random.choice(data)
            # Just compute loss on last output
            if i == j:
                loss = loss - l2loss(net(mx.nd.array(x)), net(mx.nd.array(y)))
            else:
                loss = loss + l2loss(net(mx.nd.array(x)), net(mx.nd.array(y)))
        loss.backward()
    trainer.step(BATCH_SIZE)
    avg_loss += mx.nd.mean(loss).asscalar()
    
    if (epoch+1) % 10 == 0:
        print(epoch, avg_loss/(len(tf_values) + len(mx_values))**2)
        for i in np.random.randint(len(tf_values), size=10):
            losses = []
            for j in range(len(mx_values)):
                losses.append((l2loss(net(x).reshape((1, -1)), net(y).reshape((1, -1))).asscalar(), j))
                losses = sorted(losses)[:3]
            print('\t', tf_labels[i], ' ==> ', [mx_labels[j] for l, j in losses])
    
    
    

MXNetError: [16:52:49] src/pass/gradient.cc:187: Operator _copyto is non-differentiable because it didn't register FGradient attribute.

Stack trace returned 10 entries:
[bt] (0) 0   libmxnet.so                         0x000000010e316408 _ZN4dmlc15LogMessageFatalD2Ev + 40
[bt] (1) 1   libmxnet.so                         0x000000010faac311 _ZN4nnvm4pass12_GLOBAL__N_18GradientENS_5GraphE + 15889
[bt] (2) 2   libmxnet.so                         0x000000010f278ee1 _ZNSt3__128__invoke_void_return_wrapperIN4nnvm5GraphEE6__callIJRPFS2_S2_ES2_EEES2_DpOT_ + 209
[bt] (3) 3   libmxnet.so                         0x000000010f278dd2 _ZNSt3__110__function6__funcIPFN4nnvm5GraphES3_ENS_9allocatorIS5_EES4_EclEOS3_ + 18
[bt] (4) 4   libmxnet.so                         0x000000010fa9b03b _ZN4nnvm11ApplyPassesENS_5GraphERKNSt3__16vectorINS1_12basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEENS6_IS8_EEEE + 795
[bt] (5) 5   libmxnet.so                         0x000000010f0dfddb _ZN4nnvm9ApplyPassENS_5G